In [ ]:
pip install fastapi uvicorn pandas spacy pdfplumber python-docx

In [ ]:
!pip install python-multipart
!pip install pymupdf


In [ ]:
pip install PyPDF2

In [ ]:
import pandas as pd
import PyPDF2
import re
import ast  # To parse list-like structures

# Function to extract text from PDF resume
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text + " "
    return text.strip()

# Function to extract skills from resume text
def extract_skills(text):

    import pandas as pd
    df = pd.read_csv('job_role_and_skills_required.csv')
    df['job_skills'] = df['job_skills'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
    unique_skills = set(skill.strip() for sublist in df['job_skills'] for skill in sublist)


    skill_keywords = {
        "python", "java", "sql", "machine learning", "deep learning",
        "pytorch", "javascript", "node.js", "react.js", "hadoop",
        "spark", "tableau", "power bi", "tensorflow", "nlp", "aws",
        "pandas", "numpy", "matplotlib", "seaborn", "data science", "mongodb"
    }

    skill_keywords.update(unique_skills)


    words = set(re.findall(r'\b[a-zA-Z0-9+.-]+\b', text.lower()))
    return skill_keywords.intersection(words)



# Extract skills from resume
resume_text = extract_text_from_pdf("Resume_BE PROJECT.pdf")
resume_skills = extract_skills(resume_text)

# print("Extracted Resume Skills:", resume_skills)  # Debugging output

# Read job CSV
job_df = pd.read_csv("naukri_jobsSDE.csv")

# Function to clean and extract job skills
def extract_job_skills(skill_str):
    if not isinstance(skill_str, str) or not skill_str.strip():
        return set()

    try:
        skill_data = ast.literal_eval(skill_str)  # Convert string representation of lists/sets into actual sets
        if isinstance(skill_data, list):
            skill_data = set(skill_data)
        elif not isinstance(skill_data, set):
            skill_data = {skill_data}
        return set(map(str.lower, map(str.strip, skill_data)))  # Ensure lowercase & remove spaces
    except (SyntaxError, ValueError):
        return set()  # Return empty set if parsing fails

# Apply skill extraction to CSV
job_df["Skills"] = job_df["Skills"].apply(extract_job_skills)

# Debugging: Print cleaned job skills
# print("First few job skills from CSV:\n", job_df["Skills"].head())

# Function to calculate match percentage
def calculate_match(job_skills):
    if not job_skills:
        return 0

    match_count = len(resume_skills.intersection(job_skills))
    return (match_count / len(job_skills)) * 100 if job_skills else 0

# Apply function to DataFrame
job_df["Match Percentage"] = job_df["Skills"].apply(calculate_match)

# Print results
for _, row in job_df.iterrows():
    print(f"{row['CompName']} ---- {row['Name']}: {row['Match Percentage']:.2f}% match")


Extracted Resume Skills: {'engineering', 'nlp', 'python', 'education', 'seaborn', 'scikit', 'numpy', 'tensorflow', 'react.js', 'git', 'databases', 'mysql', 'sql', 'spark', 'email', 'javascript', 'experience', 'pytorch', 'leadership', 'html', 'development', 'pyspark', 'java', 'pandas', 'prediction', 'hadoop', 'skills', 'node.js', 'matplotlib', 'teamwork'}
First few job skills from CSV:
 0    {react.js, aws, redux, sql database, socket.io...
1    {software developer, development, software, so...
2    {erp system, desktop application, desktop, ang...
3    {hibernate, plsql, spring boot, springboot, an...
4    {web development, orchestration, software deve...
Name: Skills, dtype: object
Design Centric ---- Software Developer: 12.50% match
Brandmaker333 ---- Software Developer: 25.00% match
K Raheja Realty Pvt. Ltd. ---- Software Developer: 0.00% match
In Solutions Global (ISG) ---- Software Developer: 12.50% match
Catts Apac ---- Software Developer: 0.00% match
Cisco ---- Software Engineer

In [ ]:
import pandas as pd
import PyPDF2
import re
import ast
from tabulate import tabulate  # For pretty table output

# Function to extract text from PDF resume
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text + " "
    return text.strip()

# Function to extract skills from resume text
def extract_skills(text):
    df = pd.read_csv('job_role_and_skills_required.csv')
    df['job_skills'] = df['job_skills'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
    unique_skills = set(skill.strip().lower() for sublist in df['job_skills'] for skill in sublist)

    skill_keywords = {
        "python", "java", "sql", "machine learning", "deep learning",
        "pytorch", "javascript", "node.js", "react.js", "hadoop",
        "spark", "tableau", "power bi", "tensorflow", "nlp", "aws",
        "pandas", "numpy", "matplotlib", "seaborn", "data science", "mongodb"
    }

    skill_keywords.update(unique_skills)
    words = set(re.findall(r'\b[a-zA-Z0-9+.-]+\b', text.lower()))
    return skill_keywords.intersection(words)

# Extract skills from resume
resume_text = extract_text_from_pdf("NIT_Warangal_Resume.pdf")
resume_skills = extract_skills(resume_text)

# Read job CSV
job_df = pd.read_csv("naukri_jobs_app dev.csv")

# Function to clean and extract job skills
def extract_job_skills(skill_str):
    if not isinstance(skill_str, str) or not skill_str.strip():
        return set()
    try:
        skill_data = ast.literal_eval(skill_str)
        if isinstance(skill_data, list):
            skill_data = set(skill_data)
        elif not isinstance(skill_data, set):
            skill_data = {skill_data}
        return set(map(str.lower, map(str.strip, skill_data)))
    except (SyntaxError, ValueError):
        return set()

# Apply skill extraction
job_df["Skills"] = job_df["Skills"].apply(extract_job_skills)

# Function to calculate match percentage
def calculate_match(job_skills):
    if not job_skills:
        return 0
    match_count = len(resume_skills.intersection(job_skills))
    return (match_count / len(job_skills)) * 100 if job_skills else 0

# Apply function to DataFrame
job_df["Match Percentage"] = job_df["Skills"].apply(calculate_match)

# Sort DataFrame by Match Percentage in descending order
sorted_df = job_df.sort_values(by="Match Percentage", ascending=False)

# Select relevant columns to display
display_columns = ["CompName", "Name", "Match Percentage"]
sorted_df = sorted_df[display_columns]

# Print results in tabular format
print(tabulate(sorted_df, headers="keys", tablefmt="fancy_grid", showindex=False))


╒═══════════════════════════════╤═══════════════════════════════════════════════════╤════════════════════╕
│ CompName                      │ Name                                              │   Match Percentage │
╞═══════════════════════════════╪═══════════════════════════════════════════════════╪════════════════════╡
│ The Innovative Solutions      │ APP Development Professional                      │            75      │
├───────────────────────────────┼───────────────────────────────────────────────────┼────────────────────┤
│ SP Digitech                   │ Android App Development                           │            60      │
├───────────────────────────────┼───────────────────────────────────────────────────┼────────────────────┤
│ Indiawebmediapro Technologies │ Window App Development Professional               │            40      │
├───────────────────────────────┼───────────────────────────────────────────────────┼────────────────────┤
│ Indiawebmediapro Technologies │ Pho

In [ ]:
6